<a href="https://colab.research.google.com/github/Ghxst07/SanRaksha/blob/main/Federated_Cl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flwr tensorflow
!pip install protobuf>=5.26.1,<6.0dev cryptography<44 --quiet

^C


The system cannot find the file specified.


In [2]:
!pip install --upgrade --force-reinstall "cryptography<44"
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import flwr as fl

  Using cached cryptography-43.0.3-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.4 kB)
  Using cached cffi-1.17.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached cryptography-43.0.3-cp39-abi3-manylinux_2_28_x86_64.whl (4.0 MB)
Using cached cffi-1.17.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (467 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
  Attempting uninstall: pycparser
    Found existing installation: pycparser 2.22
    Uninstalling pycparser-2.22:
      Successfully uninstalled pycparser-2.22
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
  Attempting uninstall: cryptography
    Found existing installation: cryptography 44.0.2
    Uninstalling cryptography-44.0.2:
      Successfully uninstalled cryptography-44.0.2
ERROR: pip's dependency res

In [3]:
model = Sequential()
model.add(LSTM(64,return_sequences=True))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [4]:
from re import X
class FlowerClient(fl.client.NumPyClient):
  def __init__(self,model,X_train,y_train,X_test,y_test):
    self.model = model
    self.X_train, self.y_train = X_train, y_train
    self.X_test, self.y_test = X_test, y_test

  def get_parameters(self, config):
    return self.model.get_weights()
  def fit(self,parameters,config):
    self.model.set_weights(parameters)
    self.model.fit(self.X_train,self.y_train,epoch=5,batch_size=0) #need to update batch size
    return self.model.get_weights(), len(self.X_train), {}
  def evaluate(self, parameters, config):
    self.model.set_weights(parameters)
    loss, accuracy = self.model.evaluate(self.X_test,self.y_test)
    return loss, len(self.X_test), {'accuracy':accuracy}


In [ ]:
fl.client.start_numpy_client(server_address,client=FlowerClient)